# Search and Download TSG files with CQL filter
## An interactive demonstration of accessing the AuScope Portal using the auscopecat Python library available from Pypi.
### https://pypi.org/project/auscopecat/

In [ ]:
%pip install python-slugify
%pip install -i https://test.pypi.org/simple/ auscopecat
%pip install ipyleaflet
%pip install ipywidgets
# If the above ipyleaflet pip installation does not work, you may need to try conda
#! conda install -c conda-forge ipyleaflet

In [ ]:
from auscopecat.downloadTSG import search_TSG, download_url
from IPython.display import display, Markdown
from ipyleaflet import basemaps, GeomanDrawControl, Map
import functools
import ipywidgets as widgets

AUSTRALIA_BBOX = {
    'north': -10.6681, 'east': 153.5694,
    'south': -43.6345, 'west': 113.3389
}

In [ ]:
polygon_text = widgets.Textarea(value='110.569,-10.230 155.095,-9.445 156.250,-45.161 111.027,-41.021 111.016,-41.010 110.569,-10.230',
                placeholder='Copy or paste the KML coordinates here.',
                description='Polygon:',
                layout=widgets.Layout(height="100%", width="auto"))
display(Markdown('## Enter Polygon coordinates'))
display(polygon_text)

display(Markdown('## Select Bounds'))
display(Markdown('### Enter values or draw box on the right'))

australia_btn = widgets.Button(
    description = 'Select Australia',
    button_style = '',
    tooltip = 'Reset bounds to Australia',
    icon = ''
)
display(australia_btn)

north_field = widgets.BoundedFloatText(
    description = 'North',
    value = AUSTRALIA_BBOX['north'],
    min = -90.0,
    max = 90.0,
    style = {
        'font_size': '170%'
    }
)

west_field = widgets.BoundedFloatText(
    description = 'West',
    value = AUSTRALIA_BBOX['west'],
    min = -180.0,
    max = 180.0,
    style = {
        'font_size': '170%'
    }
)

south_field = widgets.BoundedFloatText(
    description = 'South',
    value = AUSTRALIA_BBOX['south'],
    min = -90.0,
    max = 90.0,
    style = {
        'font_size': '170%'
    }
)

east_field = widgets.BoundedFloatText(
    description = 'East',
    value = AUSTRALIA_BBOX['east'],
    min = -180.0,
    max = 180.0,
    style = {
        'font_size': '170%'
    }
)
display(Markdown('\n#### Bounding Box'))
display(north_field)
display(west_field)
display(south_field)
display(east_field)

def reset_bounds(b):
    north_field.value = AUSTRALIA_BBOX['north']
    south_field.value = AUSTRALIA_BBOX['south']
    west_field.value = AUSTRALIA_BBOX['west']
    east_field.value = AUSTRALIA_BBOX['east']

australia_btn.on_click(reset_bounds)

m = Map(basemap=basemaps.Esri.WorldStreetMap, center = (-29.6, 133.0), zoom = 5)

draw_control = GeomanDrawControl()
draw_control.circlemarker = {}
draw_control.polyline = {}
draw_control.polygon = {}
draw_control.rectangle = {
    'shapeOptions': {
        'fillColor': '#fca45d',
        'color': '#fca45d',
        'fillOpacity': 0.7
    }
}

def handle_draw(target, action, geo_json):
    if action == 'create' or action == 'drag':
        coords = geo_json[0]['geometry']['coordinates'][0]
        if len(coords) == 5:
            west_field.value = str(coords[0][0])
            south_field.value = str(coords[0][1])
            east_field.value = str(coords[2][0])
            north_field.value = str(coords[2][1])
            # TODO: Enable Rectangle control (possible?)    
    elif action == 'remove':
        # TODO: Disable Rectangle control (possible?)
        pass


draw_control.on_draw(handle_draw)
m.add(draw_control)
display(m)

In [ ]:
provider_dd = widgets.Dropdown(
    options=['CSIRO', 'NSW','NT', 'QLD','SA','TAS','VIC','WA'],
    value='WA',
    description='Provider:',
)

def on_provider_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print("changed to %s" % change['new'])

provider_dd.observe(on_provider_change)

search_field = widgets.Text(
    continuous_update = False,
    placeholder = 'Enter a Borehole name and press Enter or click Search button',
    style = {
        'font_size': '170%'
    }
)
search_results_output = widgets.Output()

search_btn = widgets.Button(
    description = 'Search',
    icon = 'search',
    tooltip = 'Search'
)

clear_search_btn = widgets.Button(
    description = 'Clear',
    tooltip = 'Clear search',
    icon = 'times'
)

search_box_layout = widgets.Layout(
    display = 'flex',
    flex_flow = 'row',
    align_items = 'stretch'
)
search_box = widgets.Box(children=[provider_dd, search_field, search_btn, clear_search_btn], layout = search_box_layout)
display(Markdown('## Search For Results'))
display(search_box)
display(search_results_output)


bbox = {}
myUrls = []
def on_download_clicked(url, b):
    print(url)
    fn = url.replace('/','-').replace(':','-')
    download_url(url,fn)



def search(prov, name):
    if search_results_output:
        try:
            bbox = f'{float(west_field.value):.2f}, {float(south_field.value):.2f}, {float(east_field.value):.2f}, {float(north_field.value):.2f}'
            kmlCoords = f'{polygon_text.value}'
        except Exception as e:
            display(Markdown(f'## Error with bounds: {e}'))
            return

        try:
            print(f'{kmlCoords=}  {bbox=}  {name=}')
            urls = search_TSG(prov = prov,  name = name, bbox = bbox, kmlCoords = kmlCoords)
        except Exception as e:
            display(Markdown(f'## Error searching records: {e}'))
            return
        if len(urls) > 0:
            display(Markdown(f'\n## Results ({len(urls)}):\n'))
            button_box_layout = widgets.Layout(
                width='auto',
                display = 'flex',
                flex_flow = 'row',
                align_items = 'stretch'
            )
            
            for index, url in enumerate(urls):               
                download_btn = widgets.Button(
                    description='Download',
                    button_style='',
                    tooltip='Download TSG',
                    icon='download',
                    disabled=False
                )
                
                url_text = widgets.Text(
                    width = 'auto',
                    description=f'{index}:',
                    value=f'{url}',
                    disabled=True,
                    display='flex',
                    flex_flow='column',
                    align_items='stretch',                    
                    layout = widgets.Layout(width='auto')
                )
                download_btn.on_click(functools.partial(on_download_clicked, url))
                button_box = widgets.Box(children=[url_text, download_btn], layout = button_box_layout)
                display(button_box)
        else:
            display(Markdown('## No results were found'))

def on_search_clicked(s):
    print(f'on_search_output:event:{search_field.value}')
    search(provider_dd.value, search_field.value)
    
search_btn.on_click(on_search_clicked)

def on_clear_search_clicked(b):
    search_field.value = ''
    search_results_output.clear_output()

clear_search_btn.on_click(on_clear_search_clicked)
